## Запуск спарк, импорт

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
from pyspark.sql.window import Window
import json
import re

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab03")
         .getOrCreate())

In [3]:
spark

In [ ]:
!hdfs dfs -ls /labs/slaba03/

In [4]:
items = spark.read.csv('/labs/slaba03/laba03_items.csv', header = True, sep = '\t')
test = spark.read.csv('/labs/slaba03/laba03_test.csv', header = True, sep = ',')
train = spark.read.csv('/labs/slaba03/laba03_train.csv', header = True, sep = ',')
views = spark.read.csv('/labs/slaba03/laba03_views_programmes.csv', header = True, sep = ',')

1) laba03_train.csv содержатся факты покупки (колонка purchase) пользователями (колонка user_id) телепередач (колонка item_id). 

2) laba03_items.csv — дополнительные данные по items. Поля в файле, на которых хотелось бы остановиться:

* item_id — primary key. Соответствует item_id в предыдущем файле.
* content_type — тип телепередачи (1 — платная, 0 — бесплатная). Интересуют <b>платные</b> передачи.
* title — название передачи, текстовое поле.
* year — год выпуска передачи, число.
* genres — поле с жанрами передачи, разделёнными через запятую.

3) laba03_test.csv — тестовый датасет без указанного целевого признака purchase, который необходимо предсказать.

4) laba03_views_programmes.csv - просмотры передач с полями:

* ts_start — время начала просмотра.
* ts_end — время окончания просмотра.
* item_type — тип просматриваемого контента: live — "вживую", pvr — в записи

In [5]:
from pyspark.sql.functions import pandas_udf

In [6]:
#@pandas_udf('array<string>', PandasUDFType.SCALAR) 
def text_compile(s):
   # regex = re.split(r'\W+', re.U)
    words = s.str.split(',')
    
    return words

text_compile_udf = pandas_udf(text_compile, ArrayType(StringType()))

## Преобразование таблиц
### Views

In [7]:
views = (views
         .withColumn("start_time", F.to_timestamp(F.from_unixtime(F.col("ts_start"))))
         .withColumn("end_time", F.to_timestamp(F.from_unixtime(F.col("ts_end"))))
        )

In [8]:
views = views.withColumn("DiffInSeconds", F.col("ts_end") - F.col("ts_start"))
views =  (views
          .withColumn("DiffInMinutes",F.round(views["DiffInSeconds"]/60, 2))
          .withColumn("DiffInHours",F.round(views["DiffInSeconds"]/3600, 2)))

In [9]:
views.show(5)

+-------+-------+----------+----------+---------+-------------------+-------------------+-------------+-------------+-----------+
|user_id|item_id|  ts_start|    ts_end|item_type|         start_time|           end_time|DiffInSeconds|DiffInMinutes|DiffInHours|
+-------+-------+----------+----------+---------+-------------------+-------------------+-------------+-------------+-----------+
|      0|7101053|1491409931|1491411600|     live|2017-04-05 19:32:11|2017-04-05 20:00:00|       1669.0|        27.82|       0.46|
|      0|7101054|1491412481|1491451571|     live|2017-04-05 20:14:41|2017-04-06 07:06:11|      39090.0|        651.5|      10.86|
|      0|7101054|1491411640|1491412481|     live|2017-04-05 20:00:40|2017-04-05 20:14:41|        841.0|        14.02|       0.23|
|      0|6184414|1486191290|1486191640|     live|2017-02-04 09:54:50|2017-02-04 10:00:40|        350.0|         5.83|        0.1|
|    257|4436877|1490628499|1490630256|     live|2017-03-27 18:28:19|2017-03-27 18:57:36| 

In [10]:
views_items = items.select('item_id', 'title', 'content_type').join(views, on = 'item_id', how = 'right')

In [11]:
views_items.select('content_type').distinct().show()

+------------+
|content_type|
+------------+
|           0|
|        null|
+------------+



Во views только бесплатный контент, поэтому не использую это датафрейм далее.
Мб добавить средний просмотр?

### Items

In [12]:
items = spark.read.csv('/labs/slaba03/laba03_items.csv', header = True, sep = '\t')

In [13]:
cols_to_sel = ['item_id', 'content_type', 'title', 'year', 'genres', 'region_id']
items = items.select(cols_to_sel).filter(F.col('content_type') == 1)

In [14]:
items = items.withColumn('genres', F.regexp_replace('genres', 'Комедии', 'Комедия'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Для детей', 'Детские'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Драмы', 'Драма'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Документальные', 'Документальный'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Артхаус', 'Арт-хаус'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Боевики', 'Боевик'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Военные', 'Военный'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Исторические', 'Исторический'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Короткометражки', 'Короткометражные'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Мелодрамы', 'Мелодрама'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Музыкальные', 'Музыкальный'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Мультфильмы', 'Мультфильм'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Приключение', 'Приключения'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Семейные', 'Семейный'))
items = items.withColumn('genres', F.regexp_replace('genres', 'сказка', 'Сказка'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Сказки', 'Сказка'))
items = items.withColumn('genres', F.regexp_replace('genres', ' Сказка', 'Сказка'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Советское кино', 'Советские'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Спортивные', 'Спорт'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Триллеры', 'Триллер'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Фантастические', 'Фантастика'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Русские мультфильмы', 'Союзмультфильм'))
items = items.withColumn('genres', F.regexp_replace('genres', 'Наши', 'Русские'))

In [15]:
items = items.withColumn("genres_list", text_compile_udf(F.col("genres")))

In [16]:
items = items.withColumn('col1',F.lit('Передачи'))
items = items.withColumn("col2", text_compile_udf(F.col("col1")))
items = items.withColumn("genres_list", F.array_except("genres_list", "col2")).drop(*['col1', 'col2'])

In [17]:
items = items.select('*',F.size('genres_list').alias('genre_cnt'))
items = items.withColumn("genre_cnt_distinct", F.size(F.array_distinct("genres_list")))

## Проверка, что нет повторяющихся жанров
print(items.filter(F.col('genre_cnt_distinct')!=F.col('genre_cnt')).select('item_id', 'title', 'genres_list').count())
# СПОЙЛЕР: их нет

items = items.drop(*['genre_cnt_distinct'])

0


In [18]:
items.show(5)

+-------+------------+--------------------+------+-------+---------+-----------+---------+
|item_id|content_type|               title|  year| genres|region_id|genres_list|genre_cnt|
+-------+------------+--------------------+------+-------+---------+-----------+---------+
|  65667|           1|на пробах только ...|2013.0|Эротика|     null|  [Эротика]|        1|
|  65669|           1|скуби ду: эротиче...|2011.0|Эротика|     null|  [Эротика]|        1|
|  65668|           1|горячие девочки д...|2011.0|Эротика|     null|  [Эротика]|        1|
|  65671|           1|соблазнительницы ...|2011.0|Эротика|     null|  [Эротика]|        1|
|  65670|           1|секретные секс-ма...|2010.0|Эротика|     null|  [Эротика]|        1|
+-------+------------+--------------------+------+-------+---------+-----------+---------+
only showing top 5 rows



In [19]:
genres_df = items.select(items.item_id,F.explode(items.genres_list))

In [21]:
items.select('item_id', 'genre_cnt').groupBy('genre_cnt').count().sort('genre_cnt').show()

+---------+-----+
|genre_cnt|count|
+---------+-----+
|       -1|   33|
|        1|  244|
|        2|  822|
|        3| 1358|
|        4|  775|
|        5|  309|
|        6|  106|
|        7|   46|
|        8|   11|
+---------+-----+



### Train

In [25]:
train.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



In [28]:
train.count()

5032624

In [29]:
#Делаем панельный датафрейм с жанрами фильмов
train_exploded = train.join(genres_df, on = 'item_id', how = 'left')
train_exploded = train_exploded.withColumnRenamed('col','genre').fillna(0)

#### Обрабатываем юзеров
##### Общая статистика

In [31]:
train.select('user_id').distinct().count()

1941

In [32]:
user_purchase = train_exploded.groupBy('user_id','genre').agg(F.sum(F.col('purchase')).alias('purchase')) 
#сколько раз юзер оплачивал каждый жанр

In [33]:
windowSpec = Window.partitionBy("user_id")
user_purchase = user_purchase.withColumn('max_purch', F.max('purchase').over(windowSpec))
user_purchase = (user_purchase.join(
    train.groupBy('user_id')
    .agg(F.sum(F.col('purchase')).alias('cnt'), 
         F.mean(F.col('purchase')).alias('mean')),
    on = 'user_id'))

user_purchase = user_purchase.fillna('Без жанра', subset = ['genre'])
user_purchase = (user_purchase.withColumn('nTop', 
                                          F.rank().over(windowSpec.orderBy(F.col('purchase').desc()))
                                         )
                )

In [34]:
user_purchase.filter(F.col('cnt')==0).select('user_id').distinct().count()
#сколько юзеров вообще ничего не покупали

266

In [35]:
user_purchase.agg({'cnt': 'max'}).show()

+--------+
|max(cnt)|
+--------+
|   490.0|
+--------+



In [36]:
temp = user_purchase.select('user_id', 'cnt').distinct()

cnt = temp.agg(F.expr('percentile(cnt, array(0.25))')[0].alias('%25'),
               F.expr('percentile(cnt, array(0.5))')[0].alias('%50'),
               F.expr('percentile(cnt, array(0.75))')[0].alias('%75'),
               F.expr('percentile(cnt, array(0.9))')[0].alias('%90')
              )

cnt.show()

#Использую далее для отсечек

+---+---+---+----+
|%25|%50|%75| %90|
+---+---+---+----+
|1.0|2.0|5.0|14.0|
+---+---+---+----+



##### PIVOT
По строкам юзеры, по столбцам предпочитаемые жанры на основе покупока

In [37]:
pivotDF = train_exploded.groupBy('user_id').pivot("genre").agg(F.sum(F.col('purchase')))

In [38]:
pivotDF = pivotDF.fillna(0)
pivotDF = pivotDF.withColumnRenamed('null', 'Без жанра')

In [39]:
pivotDF = (pivotDF.join(
    train.groupBy('user_id')
    .agg(
        F.sum(F.col('purchase')).alias('cnt'), 
        F.mean(F.col('purchase')).alias('user_mean')
    ), 
    on = 'user_id'))

In [41]:
# По персентилям (рассчитаны выше) создаются столбцы характеристик как много клиент покупал
#Используется абсолют, так как доли очень низкие
pivotDF = pivotDF.withColumn('user_low', F.when(F.col('cnt') < 1, 1).otherwise(0))
pivotDF = pivotDF.withColumn('user_midlow', F.when((F.col('cnt') >= 1) & (F.col('cnt') < 2), 1).otherwise(0))
pivotDF = pivotDF.withColumn('user_mid', F.when((F.col('cnt') >= 2) & (F.col('cnt') < 5), 1).otherwise(0))
pivotDF = pivotDF.withColumn('user_midhigh', F.when((F.col('cnt') >= 5) & (F.col('cnt') < 14), 1).otherwise(0))
pivotDF = pivotDF.withColumn('user_high', F.when(F.col('cnt') >= 14, 1).otherwise(0))

In [42]:
pivotDF = pivotDF.withColumnRenamed('cnt', 'user_cnt')

In [43]:
pivotDF.filter(F.col('user_cnt')>100).select('user_id', 'user_cnt', 'user_low', 
                                        'user_midlow', 'user_mid', 'user_midhigh', 'user_high').toPandas()

,user_id,user_cnt,user_low,user_midlow,user_mid,user_midhigh,user_high
0,588378,138.0,0,0,0,0,1
1,747028,490.0,0,0,0,0,1
2,865152,116.0,0,0,0,0,1


In [44]:
#Интенсивность покупок определённых жанров
for colm in pivotDF.columns[1:len(pivotDF.columns)-7]:
    pivotDF = pivotDF.withColumn(f'{colm}_low', F.when(F.col(colm) < 3, 1).otherwise(0))
    pivotDF = pivotDF.withColumn(f'{colm}_midlow', F.when((F.col(colm) >= 3) & (F.col(colm) < 7), 1).otherwise(0))
    pivotDF = pivotDF.withColumn(f'{colm}_mid', F.when((F.col(colm) >= 7) & (F.col(colm) < 15), 1).otherwise(0))
    pivotDF = pivotDF.withColumn(f'{colm}_midhigh', F.when((F.col(colm) >= 15) & (F.col(colm) < 50), 1).otherwise(0))
    pivotDF = pivotDF.withColumn(f'{colm}_high', F.when(F.col(colm) >= 50, 1).otherwise(0))

#### Обработаем фильмы

In [46]:
item_purchase = train.groupBy('item_id').agg(F.sum(F.col('purchase')).alias('cnt'),
                                             F.mean(F.col('purchase')).alias('item_mean'))

In [47]:
temp = item_purchase.select('item_id', 'cnt').distinct()

cnt = temp.agg(F.expr('percentile(cnt, array(0.6))')[0].alias('%60'),
               F.expr('percentile(cnt, array(0.75))')[0].alias('%75'),
               F.expr('percentile(cnt, array(0.9))')[0].alias('%90')
              )

cnt.show()

+---+---+---+
|%60|%75|%90|
+---+---+---+
|1.0|2.0|7.0|
+---+---+---+



In [48]:
item_purchase = item_purchase.withColumn('item_no', F.when(F.col('cnt') < 1, 1).otherwise(0))
item_purchase = item_purchase.withColumn('item_low', F.when((F.col('cnt') >= 1) & (F.col('cnt') < 2), 1).otherwise(0))
item_purchase = item_purchase.withColumn('item_mid', F.when((F.col('cnt') >= 2) & (F.col('cnt') < 7), 1).otherwise(0))
item_purchase = item_purchase.withColumn('item_high', F.when(F.col('cnt') >= 7, 1).otherwise(0))

In [49]:
item_purchase = item_purchase.withColumnRenamed('cnt', 'item_cnt')

#### Обрабатываем жанры

In [51]:
item_genre = train_exploded.groupBy('item_id').pivot("genre").agg(F.sum(F.col('purchase')))
item_genre = item_genre.withColumnRenamed('null', 'Без жанра')

In [53]:
item_genre = (item_genre.join(
    train.groupBy('item_id').agg(
        F.sum(F.col('purchase')).alias('cnt')
    ), on = 'item_id'))

In [54]:
for genre in item_genre.columns[1:len(item_genre.columns)-1]:
    item_genre = item_genre.withColumn(genre, F.col(genre)/F.col('cnt'))

item_genre = item_genre.fillna(0)

In [55]:
item_genre = item_genre.drop(*['cnt'])
for genre in item_genre.columns[1:]:
    item_genre = item_genre.withColumnRenamed(genre, f'item_{genre}')

#### Формирование рабочего df

In [57]:
print(train.select('user_id').distinct().count())
print(train.select('item_id').distinct().count())

### для проверок

1941
3704


In [58]:
purchase_df = train.join(pivotDF, on = 'user_id')

purchase_df.select('user_id').distinct().count()

1941

In [59]:
purchase_df = purchase_df.join(item_purchase, on = 'item_id')

purchase_df.select('item_id').distinct().count()

3704

In [60]:
purchase_df = purchase_df.join(item_genre, on = 'item_id')

purchase_df.select('item_id').distinct().count()

3704

## Модель

In [62]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [63]:
###### cols = purchase_df.columns[3:]
cols = [x for x in purchase_df.columns[65:] if "_high" not in x]
cols = [x for x in cols if "_cnt" not in x]
cols = [x for x in cols if "_mean" not in x]

### Pipeline+Assembler

In [64]:
stages = []
assemblerInputs = cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [65]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(purchase_df)
purchase_df = pipelineModel.transform(purchase_df)
selectedCols = ['purchase', 'features'] +  ['user_id', 'item_id']
purchase_df = purchase_df.select(selectedCols)
purchase_df = purchase_df.withColumn('purchase', purchase_df.purchase.cast('float'))
purchase_df.printSchema()

root
 |-- purchase: float (nullable = true)
 |-- features: vector (nullable = true)
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)



### Logistic Regression
#### Train-test-split 
Для проверки алгоритма, обучала его на 70% данных трейна и соответственно тестировала на 30%

In [79]:
trainM, testM = purchase_df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(trainM.count()))
print("Test Dataset Count: " + str(testM.count()))

Training Dataset Count: 3522587
Test Dataset Count: 1510037


In [80]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'purchase', maxIter=10)
lrModel = lr.fit(trainM)

In [81]:
trainingSummary = lrModel.summary
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

Training set areaUnderROC: 0.863812746420513


In [82]:
lrPreds = lrModel.transform(testM)
lrPreds.show(5)

+--------+--------------------+-------+-------+--------------------+--------------------+----------+
|purchase|            features|user_id|item_id|       rawPrediction|         probability|prediction|
+--------+--------------------+-------+-------+--------------------+--------------------+----------+
|     0.0|(317,[0,4,8,12,16...| 651811|  74605|[6.94930158765915...|[0.99904161447376...|       0.0|
|     0.0|(317,[0,4,8,12,16...| 703514| 100140|[6.94930158765915...|[0.99904161447376...|       0.0|
|     0.0|(317,[0,4,8,12,16...| 746365|  74605|[6.94930158765915...|[0.99904161447376...|       0.0|
|     0.0|(317,[0,4,8,12,16...| 759924|  74605|[6.94930158765915...|[0.99904161447376...|       0.0|
|     0.0|(317,[0,4,8,12,16...| 765762|    691|[6.94930158765915...|[0.99904161447376...|       0.0|
+--------+--------------------+-------+-------+--------------------+--------------------+----------+
only showing top 5 rows



In [83]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lrEval = BinaryClassificationEvaluator(labelCol = 'purchase')
print('Test Area Under ROC', lrEval.evaluate(lrPreds))

Test Area Under ROC 0.8642501598161314


#### Обучение на всей train выборке

In [66]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'purchase', maxIter=10)
lrModel = lr.fit(purchase_df)

In [67]:
trainingSummary = lrModel.summary
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

Training set areaUnderROC: 0.8644187252007944


### Запускаем на тесте

In [68]:
test.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
+-------+-------+--------+
only showing top 5 rows



In [69]:
test = test.drop('purchase')

In [70]:
print(test.select('user_id').distinct().count())
print(test.select('item_id').distinct().count())

### для проверок

1941
3704


In [71]:
purchase_df_test = test.join(pivotDF, on = 'user_id')

#purchase_df_test.select('user_id').distinct().count()

In [72]:
purchase_df_test = purchase_df_test.join(item_purchase, on = 'item_id')

#purchase_df_test.select('item_id').distinct().count()

In [73]:
purchase_df_test = purchase_df_test.join(item_genre, on = 'item_id')

#purchase_df_test.select('item_id').distinct().count()

In [75]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(purchase_df_test)
purchase_df_test = pipelineModel.transform(purchase_df_test)
selectedCols = ['features'] +  ['user_id', 'item_id']
purchase_df_test = purchase_df_test.select(selectedCols)
purchase_df_test.printSchema()

root
 |-- features: vector (nullable = true)
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)



In [76]:
lrPreds = lrModel.transform(purchase_df_test)

In [77]:
predict_df = lrPreds.select('user_id', 'item_id', 'probability')

### Save

In [81]:
predict_df.write.parquet("/user/alexandra.kolesova/predict_result.parquet") 

In [4]:
predict_df = spark.read.parquet("/user/alexandra.kolesova/predict_result.parquet")

In [7]:
from pyspark.sql.types import FloatType

In [8]:
firstelement=F.udf(lambda v:float(v[1]),FloatType())

In [9]:
predict_df = predict_df.withColumn("prob_1", firstelement("probability"))

In [10]:
predict_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prob_1: float (nullable = true)



In [12]:
result = predict_df.selectExpr('user_id', 'item_id', 'prob_1 as purchase')

In [15]:
result.show(5)

+-------+-------+------------+
|user_id|item_id|    purchase|
+-------+-------+------------+
|   1654| 100026| 9.387307E-4|
|   1654| 100029|9.7587716E-4|
|   1654| 100095|0.0010013141|
|   1654| 100100| 9.680851E-4|
|   1654| 100106| 9.575816E-4|
+-------+-------+------------+
only showing top 5 rows



In [14]:
result = result.sort('user_id', 'item_id')

In [17]:
result.coalesce(1).write.options(header='True', delimiter=',').csv("/user/alexandra.kolesova/predict_result.csv") 

In [18]:
!hdfs dfs -ls /user/alexandra.kolesova

Found 3 items
drwxr-xr-x   - alexandra.kolesova alexandra.kolesova          0 2022-10-31 22:04 /user/alexandra.kolesova/.sparkStaging
drwxr-xr-x   - alexandra.kolesova alexandra.kolesova          0 2022-10-31 22:12 /user/alexandra.kolesova/predict_result.csv
drwxr-xr-x   - alexandra.kolesova alexandra.kolesova          0 2022-10-31 20:11 /user/alexandra.kolesova/predict_result.parquet


In [19]:
!hdfs dfs -copyToLocal /user/alexandra.kolesova/predict_result.csv result.csv

In [20]:
result.toPandas().to_csv('lab03.csv')

## Остановка

In [25]:
spark.stop()